In [2]:

!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/source.zip

--2024-09-05 13:26:56--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/source.zip
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.45.118.108
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.45.118.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2707 (2.6K) [application/zip]
Saving to: ‘source.zip.1’

source.zip.1        100%[===================>]   2.64K  --.-KB/s    in 0s      

2024-09-05 13:26:58 (34.5 MB/s) - ‘source.zip.1’ saved [2707/2707]



In [2]:
!unzip source.zip

/usr/bin/sh: 1: unzip: not found


In [3]:
import glob 
import pandas as pd 
import xml.etree.ElementTree as ET 
from datetime import datetime 

In [4]:
log_file = "log_file.txt" 
target_file = "transformed_data.csv" 

In [21]:

def extract_from_csv(file_to_process):
    """
    Extract data from a CSV file.
    
    Parameters:
    file_to_process (str): Path to the CSV file.
    
    Returns:
    DataFrame: Pandas DataFrame containing the data from the CSV file.
    """
    data = pd.read_csv(file_to_process)
    return data

def extract_from_json(file_to_process):
    """
    Extract data from a JSON file.
    
    Parameters:
    file_to_process (str): Path to the JSON file.
    
    Returns:
    DataFrame: Pandas DataFrame containing the data from the JSON file.
    """
    data = pd.read_json(file_to_process, lines=True)
    return data

def extract_from_xml(file_to_process):
    """
    Extract data from an XML file.
    
    Parameters:
    file_to_process (str): Path to the XML file.
    
    Returns:
    DataFrame: Pandas DataFrame containing the data from the XML file.
    """
    # Create an empty DataFrame with specified columns
    dataframe = pd.DataFrame(columns=["name", "height", "weight"])
    
    # Parse the XML file
    tree = ET.parse(file_to_process)
    root = tree.getroot()
    
    # Iterate over each person element in the XML
    for person in root:
        # Extract the name, height, and weight from the XML elements
        name = person.find("name").text
        height = float(person.find("height").text)
        weight = float(person.find("weight").text)
        
        # Append the extracted data to the DataFrame
        dataframe = pd.concat([dataframe, pd.DataFrame([[name, height, weight]], columns=["name", "height", "weight"])])
    
    return dataframe

In [22]:
def extract():
    extracted_data = pd.DataFrame(columns=["name", "height", "weight"]) 

    # Process all CSV files
    for csv_file in glob.glob("*.csv"):
        extracted_data = pd.concat([extracted_data, extract_from_csv(csv_file)], ignore_index=True)

    # Process all JSON files
    for json_file in glob.glob("*.json"):
        extracted_data = pd.concat([extracted_data, extract_from_json(json_file)], ignore_index=True)

    # Process all XML files
    for xml_file in glob.glob("*.xml"):
        extracted_data = pd.concat([extracted_data, extract_from_xml(xml_file)], ignore_index=True)

    return extracted_data

### Difference between `dataframe.apply` and `dataframe.transform`

In pandas, `apply` and `transform` are both used to perform operations on DataFrames or Series, but they have different use cases and behaviors.

#### `apply`:
- **Purpose**: Used to apply a function along an axis of the DataFrame (rows or columns).
- **Behavior**: The function can return a scalar, a Series, or a DataFrame.
- **Use Case**: When you want to apply a function to each row or column and the function can return different shapes.
- **Example**:
  ```python
  import pandas as pd

  df = pd.DataFrame({
      'A': [1, 2, 3],
      'B': [4, 5, 6]
  })

  # Apply a function to each column
  result = df.apply(lambda x: x + 1)
  print(result)

### `transform`:
- **Purpose**: Used to return a DataFrame with the same shape as the input DataFrame.
- **Behavior**: The function must return a scalar value, a Series, or an array with the same length as the input DataFrame.
- **Use Case**: When you want to transform each element in a DataFrame or Series.
- **Example**:
  ```python
  import pandas as pd

  df = pd.DataFrame({
      'A': [1, 2, 3],
      'B': [4, 5, 6]
  })

  # Transform each element in the DataFrame
  result = df.transform(lambda x: x + 1)
  print(result)
  ```

In [23]:
def transform(data):
    # Convert height from inches to meters
    data["height"] = data["height"].transform(lambda x: x * 0.0254)

    # Convert weight from pounds to kilograms
    data["weight"] = data["weight"].transform(lambda x: x * 0.453592)
    
    return data

In [24]:
def load_data(target_file, transformed_data):
    transformed_data.to_csv(target_file, index=False)

In [25]:
def log_progress(message):
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    with open(log_file, "a") as log:
        log.write(f"{timestamp} - {message}\n")

In [26]:
log_progress("ETL Job Started")

log_progress("Extracting Data")
data = extract()

log_progress("Transforming Data")
transformed_data = transform(data)

log_progress("Loading Data")
load_data(target_file, transformed_data)

log_progress("ETL Job Finished")

/tmp/ipykernel_12/3992496054.py:6: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  extracted_data = pd.concat([extracted_data, extract_from_csv(csv_file)], ignore_index=True)
/tmp/ipykernel_12/1785745781.py:52: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  dataframe = pd.concat([dataframe, pd.DataFrame([[name, height, weight]], columns=["name", "height", "weight"])])
/tmp/ipykernel_12/1785745781.py:52: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecate

In [27]:
# load the csv to pandas and show the first 5 rows
df = pd.read_csv(target_file)
df.head()

,name,height,weight
0,alex,1.670812,51.251360
1,ajay,1.816608,61.910772
2,alice,1.762760,69.413184
3,ravi,1.732788,64.564285
4,joe,1.721866,65.453326
